In [2]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
%pylab inline
from matplotlib import pyplot as plt

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


<h1>Q1: Read in data</h1>

In [24]:
gold = pd.read_csv('gold.txt',sep='\t',header=None)
gold.rename(columns={0:"url", 1:"category"}, inplace=True)
labels = pd.read_csv('labels.txt',sep='\t',header=None)
labels.rename(columns={0:"turk", 1:"url", 2:"category"}, inplace=True)

In [22]:
gold

,url,category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X
...,...,...
1512,http://zipcodez.com,G
1513,http://zml.com,P
1514,http://zonezero.com,G
1515,http://zooped.com,R


In [23]:
labels

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
...,...,...,...
92716,AFHMR93GDJCGT,http://zz868.com,G
92717,A13F119AMOA54H,http://zzx.cc,X
92718,A1S0C0EB7A6FFM,http://zzx.cc,P
92719,A21US576U8SCO4,http://zzx.cc,G


<h1>Q2: Split into two DataFrames</h1>

In [43]:
#labels_on_gold=pd.merge(labels,gold,on='url', how='inner')
labels_on_gold= labels[labels['url'].isin(gold['url'])]
labels_unknown= labels[~labels['url'].isin(gold['url'])]
labels_unknown

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
...,...,...,...
92716,AFHMR93GDJCGT,http://zz868.com,G
92717,A13F119AMOA54H,http://zzx.cc,X
92718,A1S0C0EB7A6FFM,http://zzx.cc,P
92719,A21US576U8SCO4,http://zzx.cc,G


<h1>Q3:  Compute accuracies of turks</h1>

As we need to compute count of ratings, it is a good idea to drop duplicates now

In [56]:
labels_on_gold=labels_on_gold.drop_duplicates()
labels_unknown=labels_unknown.drop_duplicates()

labels_gold=pd.merge(labels_on_gold,gold,on='url', how='inner')
label_by_turk=labels_gold.groupby(by='turk')['category_x'].count()


label_by_correct=pd.merge
label_by_turk

turk
A112DVP1KG4QZU      1
A1253FXHCZ9CWM     29
A12CY1Q7XKJJDE      1
A12RE8G66WTO8B     20
A12Y1GTGIQDGRA      3
                 ... 
AYWOBMTEGFVVI       1
AZ1S5BIG5R6P6      35
AZSDW7JJL5VA7     100
AZVB3RZW0OR1H       3
amurray            15
Name: category_x, Length: 269, dtype: int64

In [71]:
labels_gold=pd.merge(labels_on_gold,gold,on='url', how='inner')
labels_gold['flag']=labels_gold['category_x']==labels_gold['category_y']
labels_temp=labels_gold.groupby(['turk'])['flag'].agg(['count','mean']).reset_index()
rater_goodness = labels_temp.rename(columns={'count' : 'number_of_ratings', 'mean' : 'average_correctness'}).set_index('turk')
rater_goodness

,number_of_ratings,average_correctness
turk,,
A112DVP1KG4QZU,1,1.000000
A1253FXHCZ9CWM,29,0.517241
A12CY1Q7XKJJDE,1,1.000000
A12RE8G66WTO8B,20,0.750000
A12Y1GTGIQDGRA,3,0.333333
...,...,...
AYWOBMTEGFVVI,1,0.000000
AZ1S5BIG5R6P6,35,0.714286
AZSDW7JJL5VA7,100,0.680000


<h1>Q4: Odds Ratio</h1>

In [74]:
rater_goodness['odds'] = rater_goodness['average_correctness']/ (1.001-rater_goodness['average_correctness'])
rater_goodness

,number_of_ratings,average_correctness,odds
turk,,,
A112DVP1KG4QZU,1,1.000000,1000.000000
A1253FXHCZ9CWM,29,0.517241,1.069214
A12CY1Q7XKJJDE,1,1.000000,1000.000000
A12RE8G66WTO8B,20,0.750000,2.988048
A12Y1GTGIQDGRA,3,0.333333,0.499251
...,...,...,...
AYWOBMTEGFVVI,1,0.000000,0.000000
AZ1S5BIG5R6P6,35,0.714286,2.491281
AZSDW7JJL5VA7,100,0.680000,2.118380


<h1>Q5:  Most accurate turks</h1>

In [79]:
rater_goodness[rater_goodness['number_of_ratings'] >= 20].sort_values(by = 'average_correctness',ascending = False)[:10]

,number_of_ratings,average_correctness,odds
turk,,,
A2U0R4X38GUKZE,20,0.950000,18.627451
A22C0PJUBFJTI0,36,0.916667,10.869565
A23YQUBXZPKILZ,24,0.875000,6.944444
ATVALOQVDCMZW,103,0.854369,5.826657
A1HIXWH4OXT8S4,40,0.825000,4.687500
A3220HG1O83HQ4,22,0.818182,4.475385
A32W20KGQXS0LL,25,0.800000,3.980100
A20PWAB7G3HDHU,20,0.800000,3.980100
AJSJVK40F5HM6,28,0.785714,3.649635


<h1>Q6: Rating counts versus accuracy</h1>

In [109]:
rater_75=rater_goodness[rater_goodness['number_of_ratings'] > rater_goodness['number_of_ratings'].quantile(0.75)] #get turks who gave more than 75% of the ratings

reliable_turk_from_unknown=labels_unknown[labels_unknown['turk'].isin(rater_75.reset_index()['turk'])] #check for >75 quantile turks in labels_unknown
temp_turks = reliable_turk_from_unknown.merge(rater_goodness, on=['turk']) # join with the rater_goodness to bring in odds
overall_odds = temp_turks.groupby(['url','category'])['odds'].prod()
overall_odds = overall_odds.unstack() #To crosstab a Series use unstack()
#overall_odds = overall_odds.T # do we need to do this?

AttributeError: 'Series' object has no attribute 'crosstab'

In [87]:
labels_unknown

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
...,...,...,...
92716,AFHMR93GDJCGT,http://zz868.com,G
92717,A13F119AMOA54H,http://zzx.cc,X
92718,A1S0C0EB7A6FFM,http://zzx.cc,P
92719,A21US576U8SCO4,http://zzx.cc,G
